# 手写神经网络：从感知机到多层网络

**目标**: 从零实现神经网络，深入理解反向传播算法

**预计时间**: 60-90分钟

---

## 📋 本Notebook内容

1. 从零实现感知机（不使用框架）
2. 实现多层感知机（MLP）
3. 可视化激活函数
4. 手动推导并实现反向传播
5. 在XOR问题上训练并可视化决策边界

In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

# 设置中文字体和样式
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']  # macOS/Windows
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

# 设置随机种子
np.random.seed(42)

---

## 第1部分: 感知机实现

感知机是最简单的神经网络单元，只能解决线性可分问题。

In [ ]:
class Perceptron:
    """单层感知机实现"""
    
    def __init__(self, input_dim, learning_rate=0.01, epochs=100):
        """
        Args:
            input_dim: 输入特征维度
            learning_rate: 学习率
            epochs: 训练轮数
        """
        self.weights = np.zeros(input_dim)
        self.bias = 0
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.losses = []
    
    def activation(self, z):
        """阶跃激活函数: 0或1"""
        return np.where(z >= 0, 1, 0)
    
    def predict(self, X):
        """预测"""
        z = np.dot(X, self.weights) + self.bias
        return self.activation(z)
    
    def fit(self, X, y):
        """训练感知机"""
        for epoch in range(self.epochs):
            errors = 0
            for xi, yi in zip(X, y):
                # 预测
                prediction = self.predict(xi)
                
                # 更新权重（仅当预测错误时）
                error = yi - prediction
                if error != 0:
                    self.weights += self.learning_rate * error * xi
                    self.bias += self.learning_rate * error
                    errors += 1
            
            # 记录损失（错误分类数）
            self.losses.append(errors)
            
            # 收敛检查
            if errors == 0:
                print(f"收敛于epoch {epoch+1}")
                break
        
        return self

### 测试感知机：AND门

In [ ]:
# AND门数据
X_and = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_and = np.array([0, 0, 0, 1])

# 训练感知机
perceptron_and = Perceptron(input_dim=2, learning_rate=0.1, epochs=100)
perceptron_and.fit(X_and, y_and)

# 测试
print("\nAND门测试结果:")
for xi, yi in zip(X_and, y_and):
    pred = perceptron_and.predict(xi)
    print(f"输入: {xi}, 真实: {yi}, 预测: {pred}")

# 绘制损失曲线
plt.figure(figsize=(8, 4))
plt.plot(perceptron_and.losses)
plt.xlabel('Epoch')
plt.ylabel('错误分类数')
plt.title('AND门感知机训练过程')
plt.grid(True)
plt.show()

### 感知机的局限：XOR问题

XOR（异或）是线性不可分问题，单层感知机无法解决。

In [ ]:
# XOR数据
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([0, 1, 1, 0])

# 尝试训练感知机
perceptron_xor = Perceptron(input_dim=2, learning_rate=0.1, epochs=100)
perceptron_xor.fit(X_xor, y_xor)

# 测试（会失败）
print("\nXOR门测试结果:")
for xi, yi in zip(X_xor, y_xor):
    pred = perceptron_xor.predict(xi)
    print(f"输入: {xi}, 真实: {yi}, 预测: {pred}, {'✓' if pred==yi else '✗'}")

# 可视化XOR问题
plt.figure(figsize=(6, 6))
plt.scatter(X_xor[y_xor==0, 0], X_xor[y_xor==0, 1], c='blue', s=100, label='类别0', edgecolors='k')
plt.scatter(X_xor[y_xor==1, 0], X_xor[y_xor==1, 1], c='red', s=100, label='类别1', edgecolors='k')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('XOR问题：线性不可分')
plt.legend()
plt.grid(True)
plt.show()

print("\n结论: 单层感知机无法解决XOR问题，需要多层网络！")

---

## 第2部分: 激活函数可视化

激活函数引入非线性，使神经网络能够逼近复杂函数。

In [ ]:
# 定义激活函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def relu(x):
    return np.maximum(0, x)

def leaky_relu(x, alpha=0.01):
    return np.where(x > 0, x, alpha * x)

# 定义导数（用于后续反向传播）
def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

# 绘制激活函数及其导数
x = np.linspace(-5, 5, 100)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Sigmoid
axes[0, 0].plot(x, sigmoid(x), 'b-', linewidth=2)
axes[0, 0].set_title('Sigmoid')
axes[0, 0].grid(True)
axes[1, 0].plot(x, sigmoid_derivative(x), 'r-', linewidth=2)
axes[1, 0].set_title('Sigmoid导数')
axes[1, 0].grid(True)

# Tanh
axes[0, 1].plot(x, tanh(x), 'b-', linewidth=2)
axes[0, 1].set_title('Tanh')
axes[0, 1].grid(True)
axes[1, 1].plot(x, tanh_derivative(x), 'r-', linewidth=2)
axes[1, 1].set_title('Tanh导数')
axes[1, 1].grid(True)

# ReLU
axes[0, 2].plot(x, relu(x), 'b-', linewidth=2)
axes[0, 2].set_title('ReLU')
axes[0, 2].grid(True)
axes[1, 2].plot(x, relu_derivative(x), 'r-', linewidth=2)
axes[1, 2].set_title('ReLU导数')
axes[1, 2].grid(True)

# Leaky ReLU
axes[0, 3].plot(x, leaky_relu(x), 'b-', linewidth=2)
axes[0, 3].set_title('Leaky ReLU')
axes[0, 3].grid(True)
axes[1, 3].plot(x, np.where(x > 0, 1, 0.01), 'r-', linewidth=2)
axes[1, 3].set_title('Leaky ReLU导数')
axes[1, 3].grid(True)

plt.tight_layout()
plt.show()

**观察**:
- **Sigmoid**: 输出在(0,1)，但在|x|>3时梯度接近0（梯度消失）
- **Tanh**: 输出在(-1,1)，比Sigmoid好，但仍有梯度消失
- **ReLU**: 正区间梯度恒为1，负区间梯度为0（神经元死亡）
- **Leaky ReLU**: 负区间保留小梯度，缓解神经元死亡

---

## 第3部分: 多层感知机 (MLP) 实现

使用MLP解决XOR问题，手动实现反向传播。

In [ ]:
class MLP:
    """两层神经网络实现（1个隐藏层）"""
    
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.1):
        """
        Args:
            input_dim: 输入层神经元数
            hidden_dim: 隐藏层神经元数
            output_dim: 输出层神经元数
            learning_rate: 学习率
        """
        # 初始化权重（Xavier初始化）
        self.W1 = np.random.randn(input_dim, hidden_dim) * np.sqrt(2.0 / input_dim)
        self.b1 = np.zeros(hidden_dim)
        self.W2 = np.random.randn(hidden_dim, output_dim) * np.sqrt(2.0 / hidden_dim)
        self.b2 = np.zeros(output_dim)
        self.learning_rate = learning_rate
        
        # 用于存储中间值（反向传播需要）
        self.cache = {}
        self.losses = []
    
    def forward(self, X):
        """前向传播"""
        # 隐藏层
        self.cache['z1'] = np.dot(X, self.W1) + self.b1
        self.cache['a1'] = sigmoid(self.cache['z1'])
        
        # 输出层
        self.cache['z2'] = np.dot(self.cache['a1'], self.W2) + self.b2
        self.cache['a2'] = sigmoid(self.cache['z2'])
        
        return self.cache['a2']
    
    def backward(self, X, y, y_pred):
        """反向传播（手动推导）"""
        m = X.shape[0]  # 样本数
        
        # 输出层梯度
        # dL/dz2 = (y_pred - y) * sigmoid'(z2)
        dz2 = y_pred - y
        dW2 = np.dot(self.cache['a1'].T, dz2) / m
        db2 = np.sum(dz2, axis=0) / m
        
        # 隐藏层梯度
        # dL/dz1 = dL/dz2 * W2 * sigmoid'(z1)
        dz1 = np.dot(dz2, self.W2.T) * sigmoid_derivative(self.cache['z1'])
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0) / m
        
        # 更新参数
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
    
    def compute_loss(self, y, y_pred):
        """计算二元交叉熵损失"""
        m = y.shape[0]
        epsilon = 1e-8  # 防止log(0)
        loss = -np.sum(y * np.log(y_pred + epsilon) + (1 - y) * np.log(1 - y_pred + epsilon)) / m
        return loss
    
    def fit(self, X, y, epochs=10000, verbose=True):
        """训练模型"""
        y = y.reshape(-1, 1)  # 确保y的形状正确
        
        for epoch in range(epochs):
            # 前向传播
            y_pred = self.forward(X)
            
            # 计算损失
            loss = self.compute_loss(y, y_pred)
            self.losses.append(loss)
            
            # 反向传播
            self.backward(X, y, y_pred)
            
            # 打印进度
            if verbose and (epoch + 1) % 1000 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}')
        
        return self
    
    def predict(self, X):
        """预测"""
        y_pred = self.forward(X)
        return (y_pred > 0.5).astype(int)

### 在XOR问题上训练MLP

In [ ]:
# 创建并训练MLP
mlp = MLP(input_dim=2, hidden_dim=4, output_dim=1, learning_rate=0.5)
mlp.fit(X_xor, y_xor, epochs=5000, verbose=True)

# 测试
print("\n=== MLP在XOR问题上的测试结果 ===")
predictions = mlp.predict(X_xor)
for xi, yi, pred in zip(X_xor, y_xor, predictions):
    print(f"输入: {xi}, 真实: {yi}, 预测: {pred[0]}, {'✓' if pred[0]==yi else '✗'}")

# 计算准确率
accuracy = np.mean(predictions.flatten() == y_xor)
print(f"\n准确率: {accuracy:.2%}")

### 可视化训练过程

In [ ]:
# 绘制损失曲线
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(mlp.losses)
plt.xlabel('Epoch')
plt.ylabel('损失 (Binary Cross-Entropy)')
plt.title('MLP训练过程')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(mlp.losses[100:])  # 跳过前100个epoch
plt.xlabel('Epoch')
plt.ylabel('损失')
plt.title('MLP训练过程（放大）')
plt.grid(True)

plt.tight_layout()
plt.show()

### 可视化决策边界

In [ ]:
def plot_decision_boundary(model, X, y, title="决策边界"):
    """绘制决策边界"""
    # 创建网格
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    # 预测每个网格点
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # 绘制决策边界
    plt.figure(figsize=(8, 8))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap=ListedColormap(['#FFAAAA', '#AAAAFF']))
    plt.contour(xx, yy, Z, colors='k', linewidths=1, levels=[0.5])
    
    # 绘制数据点
    plt.scatter(X[y==0, 0], X[y==0, 1], c='red', s=200, label='类别0', edgecolors='k', linewidths=2)
    plt.scatter(X[y==1, 0], X[y==1, 1], c='blue', s=200, label='类别1', edgecolors='k', linewidths=2)
    
    plt.xlabel('x1', fontsize=12)
    plt.ylabel('x2', fontsize=12)
    plt.title(title, fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.show()

# 绘制XOR问题的决策边界
plot_decision_boundary(mlp, X_xor, y_xor, title="MLP在XOR问题上的决策边界")

**观察**: MLP成功学习到了非线性决策边界，将XOR问题的两类分开！

---

## 第4部分: 在螺旋数据集上测试

创建一个更复杂的非线性数据集，测试MLP的能力。

In [ ]:
def make_spiral_data(n_samples=100, n_classes=2, noise=0.2):
    """生成螺旋数据集"""
    X = np.zeros((n_samples * n_classes, 2))
    y = np.zeros(n_samples * n_classes, dtype=int)
    
    for j in range(n_classes):
        ix = range(n_samples * j, n_samples * (j + 1))
        r = np.linspace(0.0, 1, n_samples)
        t = np.linspace(j * 4, (j + 1) * 4, n_samples) + np.random.randn(n_samples) * noise
        X[ix] = np.c_[r * np.sin(t), r * np.cos(t)]
        y[ix] = j
    
    return X, y

# 生成螺旋数据
X_spiral, y_spiral = make_spiral_data(n_samples=100, n_classes=2, noise=0.15)

# 可视化数据
plt.figure(figsize=(8, 8))
plt.scatter(X_spiral[y_spiral==0, 0], X_spiral[y_spiral==0, 1], c='red', s=50, label='类别0')
plt.scatter(X_spiral[y_spiral==1, 0], X_spiral[y_spiral==1, 1], c='blue', s=50, label='类别1')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('螺旋数据集')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 训练MLP
mlp_spiral = MLP(input_dim=2, hidden_dim=20, output_dim=1, learning_rate=0.5)
mlp_spiral.fit(X_spiral, y_spiral, epochs=10000, verbose=True)

# 评估
predictions_spiral = mlp_spiral.predict(X_spiral)
accuracy_spiral = np.mean(predictions_spiral.flatten() == y_spiral)
print(f"\n螺旋数据集准确率: {accuracy_spiral:.2%}")

# 绘制决策边界
plot_decision_boundary(mlp_spiral, X_spiral, y_spiral, title="MLP在螺旋数据集上的决策边界")

---

## 第5部分: 实验不同激活函数

对比Sigmoid、Tanh、ReLU在相同任务上的表现。

In [ ]:
# 修改MLP类以支持不同激活函数
class MLPWithActivation:
    """支持不同激活函数的MLP"""
    
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.1, activation='sigmoid'):
        self.W1 = np.random.randn(input_dim, hidden_dim) * np.sqrt(2.0 / input_dim)
        self.b1 = np.zeros(hidden_dim)
        self.W2 = np.random.randn(hidden_dim, output_dim) * np.sqrt(2.0 / hidden_dim)
        self.b2 = np.zeros(output_dim)
        self.learning_rate = learning_rate
        self.activation = activation
        self.cache = {}
        self.losses = []
    
    def activate(self, z):
        if self.activation == 'sigmoid':
            return sigmoid(z)
        elif self.activation == 'tanh':
            return tanh(z)
        elif self.activation == 'relu':
            return relu(z)
    
    def activate_derivative(self, z):
        if self.activation == 'sigmoid':
            return sigmoid_derivative(z)
        elif self.activation == 'tanh':
            return tanh_derivative(z)
        elif self.activation == 'relu':
            return relu_derivative(z)
    
    def forward(self, X):
        self.cache['z1'] = np.dot(X, self.W1) + self.b1
        self.cache['a1'] = self.activate(self.cache['z1'])
        self.cache['z2'] = np.dot(self.cache['a1'], self.W2) + self.b2
        self.cache['a2'] = sigmoid(self.cache['z2'])  # 输出层始终用sigmoid
        return self.cache['a2']
    
    def backward(self, X, y, y_pred):
        m = X.shape[0]
        dz2 = y_pred - y
        dW2 = np.dot(self.cache['a1'].T, dz2) / m
        db2 = np.sum(dz2, axis=0) / m
        
        dz1 = np.dot(dz2, self.W2.T) * self.activate_derivative(self.cache['z1'])
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0) / m
        
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
    
    def compute_loss(self, y, y_pred):
        m = y.shape[0]
        epsilon = 1e-8
        loss = -np.sum(y * np.log(y_pred + epsilon) + (1 - y) * np.log(1 - y_pred + epsilon)) / m
        return loss
    
    def fit(self, X, y, epochs=5000, verbose=False):
        y = y.reshape(-1, 1)
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = self.compute_loss(y, y_pred)
            self.losses.append(loss)
            self.backward(X, y, y_pred)
            if verbose and (epoch + 1) % 1000 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}')
        return self
    
    def predict(self, X):
        y_pred = self.forward(X)
        return (y_pred > 0.5).astype(int)

# 对比不同激活函数
activations = ['sigmoid', 'tanh', 'relu']
results = {}

for act in activations:
    print(f"\n=== 训练使用{act}激活函数的MLP ===")
    model = MLPWithActivation(input_dim=2, hidden_dim=10, output_dim=1, 
                              learning_rate=0.5, activation=act)
    model.fit(X_xor, y_xor, epochs=5000, verbose=False)
    
    predictions = model.predict(X_xor)
    accuracy = np.mean(predictions.flatten() == y_xor)
    results[act] = {'model': model, 'accuracy': accuracy}
    print(f"{act}准确率: {accuracy:.2%}")

# 绘制损失曲线对比
plt.figure(figsize=(12, 4))
for i, act in enumerate(activations):
    plt.subplot(1, 3, i+1)
    plt.plot(results[act]['model'].losses)
    plt.title(f'{act} (准确率: {results[act]["accuracy"]:.0%})')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
plt.tight_layout()
plt.show()

---

## 📝 总结

通过本Notebook，我们：

1. ✅ **从零实现了感知机**，理解了它的局限性（只能解决线性可分问题）
2. ✅ **手动推导并实现了反向传播算法**，深入理解梯度计算过程
3. ✅ **实现了MLP**，成功解决了XOR问题和螺旋数据集
4. ✅ **可视化了不同激活函数**的特性及其导数
5. ✅ **对比了不同激活函数**在相同任务上的表现

### 关键要点

- **单层感知机**: 只能解决线性可分问题
- **MLP**: 通过增加隐藏层引入非线性，可以逼近任意函数
- **反向传播**: 使用链式法则计算梯度，是训练神经网络的核心算法
- **激活函数**: ReLU在大多数情况下是默认选择（计算快、缓解梯度消失）

### 下一步

- 📘 继续学习 `02-pytorch-basics.ipynb`，使用PyTorch重新实现上述内容
- 🎯 尝试调整超参数（隐藏层大小、学习率、激活函数）观察效果
- 📚 深入阅读 Deep Learning Book 第6章：深度前馈网络